<a href="https://colab.research.google.com/github/monishramadoss/ofa_quant/blob/main/ofa_quant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch==1.4.0 torchvision==0.5.0
!pip install ofa
! git clone https://github.com/seshuad/IMagenet
! ls 'IMagenet/tiny-imagenet-200/'

     |████████████████████████████████| 753.4 MB 6.7 kB/s 
     |████████████████████████████████| 4.0 MB 44.8 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.
     |████████████████████████████████| 102 kB 9.6 MB/s 
Cloning into 'IMagenet'...
remote: Enumerating objects: 120594, done.
remote: Total 120594 (delta 0), re

In [2]:
import time
import skimage.io as nd
import numpy as np
import torch
import skimage.color


path = 'IMagenet/tiny-imagenet-200/'

def get_id_dictionary():
    id_dict = {}
    for i, line in enumerate(open( path + 'wnids.txt', 'r')):
        id_dict[line.replace('\n', '')] = i
    return id_dict
  
def get_class_to_id_dict():
    id_dict = get_id_dictionary()
    all_classes = {}
    result = {}
    for i, line in enumerate(open( path + 'words.txt', 'r')):
        n_id, word = line.split('\t')[:2]
        all_classes[n_id] = word
    for key, value in id_dict.items():
        result[value] = (key, all_classes[key])      
    return result

def get_data(id_dict):
    print('starting loading data')
    train_data, test_data = [], []
    train_labels, test_labels = [], []
    t = time.time()
    for key, value in id_dict.items():
        train_data += [nd.imread( path + 'train/{}/images/{}_{}.JPEG'.format(key, key, str(i)), as_gray=False ) for i in range(500)]
        train_labels_ = np.array([[0]*200]*500)
        train_labels_[:, value] = 1
        train_labels += train_labels_.tolist()

    for line in open( path + 'val/val_annotations.txt'):
        img_name, class_id = line.split('\t')[:2]
        test_data += [nd.imread( path + 'val/images/{}'.format(img_name), as_gray=False)]
        
        # test_data.append(test_data_)
        test_labels_ = np.array([[0]*200])
        test_labels_[0, id_dict[class_id]] = 1
        test_labels += test_labels_.tolist()

    for i in range(len(test_data)):
        if test_data[i].ndim == 2:
            test_data[i] = skimage.color.gray2rgb(test_data[i])

    for i in range(len(train_data)):
        if train_data[i].ndim == 2:
            train_data[i] = skimage.color.gray2rgb(train_data[i])

    print('finished loading data, in {} seconds'.format(time.time() - t))
    test = np.array(test_data), np.array(test_labels)  
    return np.array(train_data), np.array(train_labels), *test

def evaluate(model, data, target):
    model.eval()  # set model in eval mode
    total_time = 0
    num_correct = 0  # total 1000
    with torch.no_grad():
        for image, target in zip(data, target):
            # print(data[0].shape)
            start = time.time()
            result = model(image)
            total_time += time.time() - start
            
            prediction = idx2label[int(result[0].sort()[1][-1:])]
            if target == prediction:
                num_correct += 1
    
    inference_time = total_time / len(data)
    accuracy = num_correct / len(data)
    return inference_time, accuracy

train_data, train_labels, test_data, train_labels = get_data(get_id_dictionary())

starting loading data
finished loading data, in 50.535794496536255 seconds


In [3]:
print(train_data.shape, train_labels.shape)

(100000, 64, 64, 3) (100000, 200)


In [4]:
import copy
import torch
from ofa.model_zoo import ofa_net
from ofa.imagenet_classification.data_providers.imagenet import ImagenetDataProvider
# from ofa.imagenet_classification.run_manager import ImagenetRunConfig, RunManager

raw_resnet = ofa_net('ofa_resnet50', pretrained=True)
raw_resnet.set_max_net()
max_subnet = raw_resnet.get_active_subnet(preserve_weight=True)
raw_resnet.set_active_subnet(d=0, w=0, e=0)
min_subnet = raw_resnet.get_active_subnet(preserve_weight=True)


# ImagenetDataProvider.DEFAULT_PATH = path
# run_config = ImagenetRunConfig(test_batch_size=16, n_worker=20)
# run_config.data_provider.assign_active_img_size(64)


# run_manager = RunManager('./tmp/eval_subnet', min_subnet, run_config, init=False)
# run_manager.reset_running_statistics(net=min_subnet)

Downloading: "https://hanlab.mit.edu/files/OnceForAll/ofa_nets/ofa_resnet50_d=0+1+2_e=0.2+0.25+0.35_w=0.65+0.8+1.0" to .torch/ofa_nets/ofa_resnet50_d=0+1+2_e=0.2+0.25+0.35_w=0.65+0.8+1.0


In [5]:
# loss, (top1, top5) = run_manager.validate(net=max_subnet, is_test=True)
# print('\nResults: loss=%.5f,\t top1=%.1f,\t top5=%.1f' % (loss, top1, top5))
# loss, (top1, top5) = run_manager.validate(net=min_subnet, is_test=True)
# print('\nResults: loss=%.5f,\t top1=%.1f,\t top5=%.1f' % (loss, top1, top5))

In [6]:
from ofa.imagenet_classification.networks import ResNets
from ofa.utils.layers import IdentityLayer, ResidualBlock

def val2list(val, repeat_time=1):
    if isinstance(val, list) or isinstance(val, np.ndarray):
        return val
    elif isinstance(val, tuple):
        return list(val)
    else:
        return [val for _ in range(repeat_time)]

def set_active_subnet(ofa, d=None, e=None, w=None, **kwargs):
    depth = val2list(d, len(ofa.BASE_DEPTH_LIST) + 1)
    expand_ratio = val2list(e, len(ofa.blocks))
    width_mult = val2list(w, len(ofa.BASE_DEPTH_LIST) + 2)
    for block, e in zip(ofa.blocks, expand_ratio):
        if e is not None:
            block.active_expand_ratio = e

    if width_mult[0] is not None:
        ofa.input_stem[1].conv.active_out_channel = ofa.input_stem[0].active_out_channel = \
            ofa.input_stem[0].out_channel_list[width_mult[0]]
    if width_mult[1] is not None:
        ofa.input_stem[2].active_out_channel = ofa.input_stem[2].out_channel_list[width_mult[1]]

    if depth[0] is not None:
        ofa.input_stem_skipping = (depth[0] != max(ofa.depth_list))
    for stage_id, (block_idx, d, w) in enumerate(zip(ofa.grouped_block_index, depth[1:], width_mult[2:])):
        if d is not None:
            ofa.runtime_depth[stage_id] = max(ofa.depth_list) - d
        if w is not None:
            for idx in block_idx:
                ofa.blocks[idx].active_out_channel = ofa.blocks[idx].out_channel_list[w]

def set_max_subnet(ofa):
    set_active_subnet(ofa, max(ofa.depth_list), max(ofa.expand_ratio_list), len(ofa.width_mult_list) - 1)

def get_active_subnet(ofa, preserve_weight=True):
    input_stem = [ofa.input_stem[0].get_active_subnet(3, preserve_weight)]
    active_out = ofa.input_stem[0].active_out_channel
    input_stem_blocks = [0]

    if ofa.input_stem_skipping <= 0:        
        input_stem.append(ResidualBlock(
            ofa.input_stem[1].conv.get_active_subnet(active_out, preserve_weight),
            IdentityLayer(active_out, active_out)
        ))
        input_stem_blocks += [1]
    input_stem_blocks += [2]
    input_stem.append(ofa.input_stem[2].get_active_subnet(active_out, preserve_weight))
    input_channel = ofa.input_stem[2].active_out_channel

    blocks = []
    block_groups = []
    for stage_id, block_idx in enumerate(ofa.grouped_block_index):
        depth_param = ofa.runtime_depth[stage_id]
        active_idx = block_idx[:len(block_idx) - depth_param]
        block_groups.append(active_idx)
        for idx in active_idx:
            blocks.append(ofa.blocks[idx].get_active_subnet(input_channel, preserve_weight))
            input_channel = ofa.blocks[idx].active_out_channel
    classifier = ofa.classifier.get_active_subnet(input_channel, preserve_weight)

    subnet = ResNets(input_stem, blocks, classifier)
    subnet.set_bn_param(**ofa.get_bn_param())
    subnet.input_stem_blocks = input_stem_blocks
    subnet.block_groups = block_groups
    return subnet


In [7]:
# run_manager_quant = RunManager('./tmp/eval', min_subnet, run_config=run_config, init=False, no_gpu=True)

In [19]:
input_fp32 = torch.randn(1, 3, 224, 224)

set_max_subnet(raw_resnet)
max_subnet = get_active_subnet(raw_resnet)

max_subnet.eval()
config = 'fbgemm'
max_subnet.qconfig = torch.quantization.get_default_qconfig(config)
max_subnet_1 = torch.quantization.prepare(max_subnet)
_ = max_subnet_1(input_fp32)
quat_max_model = torch.quantization.convert(max_subnet_1)




ConvLayer(
  (conv): QuantizedConv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), scale=0.35585781931877136, zero_point=62, padding=(1, 1), bias=False)
  (bn): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act): QuantizedReLU(inplace=True)
)

In [ ]:
temp = dict({('blocks.0.conv1.conv.scale', 0.3034),
 ('blocks.0.conv2.conv.scale', 0.3898),
 ('blocks.0.conv3.conv.scale', 0.3047),
 ('blocks.0.downsample.conv.scale', 0.4795),
 ('blocks.1.conv1.conv.scale', 0.2099),
 ('blocks.1.conv2.conv.scale', 0.2258),
 ('blocks.1.conv3.conv.scale', 0.3333),
 ('blocks.10.conv1.conv.scale', 0.2817),
 ('blocks.10.conv2.conv.scale', 0.3093),
 ('blocks.10.conv3.conv.scale', 0.2963),
 ('blocks.11.conv1.conv.scale', 0.6109),
 ('blocks.11.conv2.conv.scale', 0.5077),
 ('blocks.11.conv3.conv.scale', 0.3028),
 ('blocks.12.conv1.conv.scale', 0.2717),
 ('blocks.12.conv2.conv.scale', 0.2179),
 ('blocks.12.conv3.conv.scale', 0.1239),
 ('blocks.13.conv1.conv.scale', 0.2204),
 ('blocks.13.conv2.conv.scale', 0.4078),
 ('blocks.13.conv3.conv.scale', 0.3996),
 ('blocks.14.conv1.conv.scale', 0.2612),
 ('blocks.14.conv2.conv.scale', 0.2186),
 ('blocks.14.conv3.conv.scale', 0.3380),
 ('blocks.14.downsample.conv.scale', 0.5858),
 ('blocks.15.conv1.conv.scale', 0.7731),
 ('blocks.15.conv2.conv.scale', 0.4752),
 ('blocks.15.conv3.conv.scale', 0.5223),
 ('blocks.16.conv1.conv.scale', 0.3974),
 ('blocks.16.conv2.conv.scale', 0.2534),
 ('blocks.16.conv3.conv.scale', 0.3723),
 ('blocks.17.conv1.conv.scale', 0.2302),
 ('blocks.17.conv2.conv.scale', 0.2933),
 ('blocks.17.conv3.conv.scale', 0.3231),
 ('blocks.2.conv1.conv.scale', 0.1517),
 ('blocks.2.conv2.conv.scale', 0.1271),
 ('blocks.2.conv3.conv.scale', 0.1200),
 ('blocks.3.conv1.conv.scale', 0.1193),
 ('blocks.3.conv2.conv.scale', 0.1338),
 ('blocks.3.conv3.conv.scale', 0.0980),
 ('blocks.4.conv1.conv.scale', 0.2068),
 ('blocks.4.conv2.conv.scale', 0.3214),
 ('blocks.4.conv3.conv.scale', 0.4915),
 ('blocks.4.downsample.conv.scale', 0.2356),
 ('blocks.5.conv1.conv.scale', 0.1636),
 ('blocks.5.conv2.conv.scale', 0.3325),
 ('blocks.5.conv3.conv.scale', 0.3767),
 ('blocks.6.conv1.conv.scale', 0.2506),
 ('blocks.6.conv2.conv.scale', 0.2422),
 ('blocks.6.conv3.conv.scale', 0.1694),
 ('blocks.7.conv1.conv.scale', 0.2284),
 ('blocks.7.conv2.conv.scale', 0.3655),
 ('blocks.7.conv3.conv.scale', 0.3437),
 ('blocks.8.conv1.conv.scale', 0.3289),
 ('blocks.8.conv2.conv.scale', 0.3125),
 ('blocks.8.conv3.conv.scale', 0.3898),
 ('blocks.8.downsample.conv.scale', 0.5282),
 ('blocks.9.conv1.conv.scale', 0.3315),
 ('blocks.9.conv2.conv.scale', 0.3270),
 ('blocks.9.conv3.conv.scale', 0.2874),
 ('input_stem.0.conv.scale', 0.6389),
 ('input_stem.1.conv.conv.scale', 0.5264),
 ('input_stem.2.conv.scale', 0.5124)})

In [64]:
import torch.nn as nn
def nested_children(m: torch.nn.Module):
    children = dict(m.named_children())
    output = {}
    if children == {}:
        # if module has no children; m is last child! :O
        return m
    else:
        # look for children from children... to the last child!
        for name, child in children.items():
            try:
                output[name] = nested_children(child)
            except TypeError:
                output[name] = nested_children(child)
    return output

def squash_nested_dict(nested_dict, ret_lst={}, prefix='', mod=None):
    if nested_dict == {}:
        ret_lst[prefix[1:]] = mod
        return
    for k in nested_dict.keys():
        if isinstance(nested_dict[k], dict):
            squash_nested_dict(nested_dict[k], ret_lst, prefix+'.'+k)
        else:
            squash_nested_dict({}, ret_lst, prefix+'.'+k, nested_dict[k])



class Quant_Model(nn.Module):
    def __init__(self, quat_model, float_model):
        super(Quant_Model, self).__init__()
        self.input_zero_points = {}
        self.input_scales = {}
        self.float_model = float_model
        self.quat_model = quat_model
        self.quant_state_dict = quat_model.state_dict()

        layer_names = {}
        named_layers = nested_children(self.float_model)
        squash_nested_dict(named_layers, layer_names)
        
        for i, l in enumerate(list(layer_names.keys())):
            layer_names[l].register_forward_pre_hook(self.forward_pre_hook(l))

    def forward_pre_hook(self, layer_name):
        def pre_hook(module, x):
            if isinstance(module, nn.Conv2d) or isinstance(module, nn.Identity):
                with torch.no_grad():
                    zero_point = self.quant_state_dict[layer_name+'.zero_point'].float()
                    scale = self.quant_state_dict[layer_name+'.scale'].float()
                    quant_min = 0.0
                    quant_max = 1.0
                    tmp = torch.clamp(torch.round(torch.div(x[0], scale) + zero_point), quant_min, quant_max) - zero_point
                    x = tmp*scale
            return x
        return pre_hook

    def forward(self, x):
        return self.float_model(x)


set_active_subnet(raw_resnet, 0, 0, 0)
min_subnet = get_active_subnet(raw_resnet)
min_subnet.eval()
min_subnet.qconfig = torch.quantization.get_default_qconfig(config)

input_fp32 = torch.randn(1, 3, 224, 224, dtype=torch.float32)
min_subnet_1 = torch.quantization.prepare(min_subnet)

_ = min_subnet_1(input_fp32)
quat_min_model = torch.quantization.convert(min_subnet_1)

fake_quant_model = Quant_Model(quat_min_model, min_subnet)
fake_output = fake_quant_model(input_fp32)
print(fake_output)

tensor([[-2.9306e+02, -6.4418e+00, -5.3657e+01, -2.2284e+01,  2.2735e+02,
         -1.1329e+02,  4.8487e+01,  5.6451e+01,  1.7572e+02,  3.6831e+01,
         -4.4456e+01, -8.1882e+01,  2.7150e+01,  5.7271e+01, -1.7678e+01,
         -7.3381e+01, -3.7374e+01, -4.1221e+01, -2.2437e+01, -4.5825e+01,
         -1.1655e+02,  2.2238e+01, -2.1023e+02,  3.3650e+01,  6.6236e+01,
         -6.9066e+01, -5.9876e+01, -4.0418e+01, -2.2419e+01, -1.0336e+02,
          1.9451e+02, -9.6390e+01,  5.4402e+01, -5.4740e+01, -5.9053e+01,
          6.0623e+01,  1.3135e+02,  7.9664e+01, -1.6394e+02,  1.1328e+02,
         -4.9874e+01,  6.3055e+01,  5.7795e+01,  1.4287e+02,  7.2237e+01,
         -6.0681e+00, -4.6383e+01,  8.5416e+01,  1.0007e+02, -2.3468e+01,
          1.1398e+02,  1.9358e+01, -1.0284e+02, -5.4639e+01,  1.5209e+00,
         -1.4771e+02, -1.5485e+02,  8.2363e+01,  3.2800e+01, -6.5999e+01,
         -8.1575e+01, -1.2488e+02,  1.2206e+02,  7.5755e+01, -9.8413e+01,
          9.9465e+01, -1.2350e+02, -3.

In [ ]:


fake_quant_model = Quant_Model(quat_min_model, torch.load('./weight_after_ps.pth'))
fake_output = fake_quant_model(input_fp32)
print(fake_output)